<a href="https://colab.research.google.com/github/Nocimaa/Binance_Bot/blob/main/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget "https://raw.githubusercontent.com/Nocimaa/Binance_Bot/refs/heads/main/btc_1h_adx_2017-08-17_to_2025-09-17(in).csv"

--2025-09-19 12:43:09--  https://raw.githubusercontent.com/Nocimaa/Binance_Bot/refs/heads/main/btc_1h_adx_2017-08-17_to_2025-09-17(in).csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28620838 (27M) [text/plain]
Saving to: ‘btc_1h_adx_2017-08-17_to_2025-09-17(in).csv’

btc_1h_adx_2017-08- 100%[===================>]  27.29M  --.-KB/s    in 0.1s    

2025-09-19 12:43:10 (284 MB/s) - ‘btc_1h_adx_2017-08-17_to_2025-09-17(in).csv’ saved [28620838/28620838]



In [2]:
import pandas as pd
import numpy as np

#Test

In [3]:
base = pd.read_csv("btc_1h_adx_2017-08-17_to_2025-09-17(in).csv")

df = base[base["timestamp"] < "2025-09-01 00:00:00+02:00"]
predict_df = base[base["timestamp"] >= "2025-09-01 00:00:00+02:00"]

def max_log_return(close, high, low):
    log_high = np.log(high / close)
    log_low = np.log(low / close)
    return log_high if abs(log_high) > abs(log_low) else log_low

def preprocessing(df, index_label=True):
  if index_label:
    # df["label"] = np.log()
    df["label"] = [
      max_log_return(c, h, l)
      for c, h, l in zip(df["close"], df["high"].shift(-1), df["low"].shift(-1))
    ]
  df = df.drop("timestamp", axis=1)
  # df = df.drop("close", axis=1)


  features = ["open","high","low","close","volume","ATR14","MACD"]
  window = 10
  for f in features:
      for w in range(1, window+1):
          df[f"{f}_lag{w}"] = df[f].shift(w)


  df.dropna(inplace=True)
  return df

df = preprocessing(df)
predict_df = preprocessing(predict_df, index_label=False)

/tmp/ipykernel_19/3276618716.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["label"] = [


In [4]:
df.head()
df["label"]

10      -0.024551
11       0.030981
12      -0.019319
13       0.011229
14      -0.007572
           ...   
70249    0.000652
70250    0.002233
70251   -0.002113
70252    0.001724
70253   -0.001475
Name: label, Length: 70244, dtype: float64

In [5]:
y = df["label"]
X = df.drop("label", axis=1)

In [6]:
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.preprocessing import StandardScaler

k = 30
selector = SelectKBest(score_func=f_regression, k=k)
X_new = selector.fit_transform(X, y)

scaler = StandardScaler()
X_new = scaler.fit_transform(X_new)

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.2, random_state=42)

In [8]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



def create_sequences(X, y, seq_len=50):
    Xs, ys = [], []
    for i in range(len(X) - seq_len):
        Xs.append(X[i:i+seq_len])
        ys.append(y[i+seq_len])
    return np.array(Xs), np.array(ys)


y_scaler = StandardScaler()
y_train_scaled = y_scaler.fit_transform(y_train.values.reshape(-1,1))
y_test_scaled  = y_scaler.transform(y_test.values.reshape(-1,1))

seq_len = 20  # tu peux ajuster

X_train_seq, y_train_seq = create_sequences(X_train, y_train_scaled, seq_len)
X_test_seq, y_test_seq   = create_sequences(X_test,  y_test_scaled,  seq_len)


X_train_tensor = torch.tensor(X_train_seq, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train_seq, dtype=torch.float32).to(device)

X_test_tensor  = torch.tensor(X_test_seq, dtype=torch.float32).to(device)
y_test_tensor  = torch.tensor(y_test_seq, dtype=torch.float32).to(device)

In [9]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class LSTMNet(nn.Module):
    def __init__(self, input_dim, hidden_dim=64, num_layers=3, dropout=0.25):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        # LSTM empilé
        self.lstm = nn.LSTM(
            input_dim,
            hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout
        )

        # Normalisation + Fully Connected
        self.bn = nn.BatchNorm1d(hidden_dim)
        self.fc1 = nn.Linear(hidden_dim, 64)
        self.fc2 = nn.Linear(64, 1)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # x: (batch, seq_len, input_dim

        out, _ = self.lstm(x)  # out: (batch, seq_len, hidden_dim)
        out = out[:, -1, :]    # garder le dernier état
        out = self.bn(out)
        out = F.relu(self.fc1(out))
        out = self.dropout(out)
        return self.fc2(out)


model = LSTMNet(input_dim=k)
model = model.to(device)

In [10]:
criterion = nn.SmoothL1Loss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 100
batch_size = 256

In [11]:
gradScaler = torch.amp.GradScaler() 


for epoch in range(epochs):
    permutation = torch.randperm(X_train_tensor.size(0))
    model.train()
    
    for i in range(0, X_train_tensor.size(0), batch_size):
        indices = permutation[i:i+batch_size]
        batch_x, batch_y = X_train_tensor[indices], y_train_tensor[indices]

        # Déplacer les batchs sur le device
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)

        optimizer.zero_grad()

        # Mixed precision
        with torch.amp.autocast('cuda'):
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)

        # Backprop avec GradScaler
        gradScaler.scale(loss).backward()

        # Gradient clipping (optionnel)
        #torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        gradScaler.step(optimizer)
        gradScaler.update()

        # Libération mémoire intermédiaire
        del outputs, loss, batch_x, batch_y
        if device == "cuda":
            torch.cuda.empty_cache()

    # Évaluation train
    model.eval()
    with torch.no_grad():
        X_train_device = X_train_tensor.to(device)
        y_train_device = y_train_tensor.to(device)
        preds = model(X_train_device)
        mse = criterion(preds, y_train_device).item()

    print(f"Epoch {epoch+1}/{epochs}, Train MSE={mse:.4f}")

Epoch 1/100, Train MSE=0.3282
Epoch 2/100, Train MSE=0.3281
Epoch 3/100, Train MSE=0.3279
Epoch 4/100, Train MSE=0.3278
Epoch 5/100, Train MSE=0.3278
Epoch 6/100, Train MSE=0.3278
Epoch 7/100, Train MSE=0.3276
Epoch 8/100, Train MSE=0.3277
Epoch 9/100, Train MSE=0.3276
Epoch 10/100, Train MSE=0.3276
Epoch 11/100, Train MSE=0.3274
Epoch 12/100, Train MSE=0.3273
Epoch 13/100, Train MSE=0.3271
Epoch 14/100, Train MSE=0.3267
Epoch 15/100, Train MSE=0.3264
Epoch 16/100, Train MSE=0.3259
Epoch 17/100, Train MSE=0.3254
Epoch 18/100, Train MSE=0.3237
Epoch 19/100, Train MSE=0.3236
Epoch 20/100, Train MSE=0.3223
Epoch 21/100, Train MSE=0.3228
Epoch 22/100, Train MSE=0.3208
Epoch 23/100, Train MSE=0.3187
Epoch 24/100, Train MSE=0.3182
Epoch 25/100, Train MSE=0.3161
Epoch 26/100, Train MSE=0.3156
Epoch 27/100, Train MSE=0.3146
Epoch 28/100, Train MSE=0.3115
Epoch 29/100, Train MSE=0.3100
Epoch 30/100, Train MSE=0.3083
Epoch 31/100, Train MSE=0.3067
Epoch 32/100, Train MSE=0.3064
Epoch 33/100, Tra

In [12]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
import torch

with torch.no_grad():
    # prédictions sur GPU
    preds = model(X_test_tensor)

    # ramener sur CPU avant conversion en numpy
    preds_cpu = preds.cpu().numpy()
    y_true_cpu = y_test_tensor.cpu().numpy()

    # si tu as utilisé un scaler pour y
    preds_orig = y_scaler.inverse_transform(preds_cpu)
    y_true_orig = y_scaler.inverse_transform(y_true_cpu)

    # calcul métriques
    from sklearn.metrics import mean_squared_error, mean_absolute_error
    import numpy as np

    mse_test = mean_squared_error(y_true_orig, preds_orig)
    rmse_test = np.sqrt(mse_test)
    mae_test = mean_absolute_error(y_true_orig, preds_orig)

print(f"\nTest MSE (original scale) = {mse_test:.4f}")
print(f"Test RMSE (original scale) = {rmse_test:.4f}")
print(f"Test MAE (original scale)  = {mae_test:.4f}")




Test MSE (original scale) = 0.0002
Test RMSE (original scale) = 0.0127
Test MAE (original scale)  = 0.0087


In [13]:
def predict_new_row(seq_rows, model, selector, scaler_X, scaler_y, feature_columns, device='cpu'):
    """
    Prédit le label à partir d'une séquence de nouvelles données (LSTM).

    Args:
        seq_rows (list[pd.Series] ou pd.DataFrame): séquence de lignes (longueur = seq_len)
        model (torch.nn.Module): modèle PyTorch déjà entraîné
        selector (sklearn transformer): SelectKBest ou autre
        scaler_X (StandardScaler): scaler pour X
        scaler_y (StandardScaler): scaler pour y
        feature_columns (list): noms des colonnes/features utilisées
        device (str): 'cpu' ou 'cuda'

    Returns:
        float: prédiction en échelle originale
    """
    # 1️⃣ Construire DataFrame à partir de la séquence
    if isinstance(seq_rows, list):
        df_seq = pd.DataFrame(seq_rows, columns=feature_columns)
    elif isinstance(seq_rows, pd.Series):
        df_seq = pd.DataFrame([seq_rows], columns=feature_columns)
    else:
        df_seq = seq_rows.copy()

    # 2️⃣ Supprimer colonne timestamp si présente
    if "timestamp" in df_seq.columns:
        df_seq = df_seq.drop(columns=["timestamp"])

    # 3️⃣ Appliquer la sélection de features
    X_new = selector.transform(df_seq)

    # 4️⃣ Normaliser
    X_new_scaled = scaler_X.transform(X_new)

    # 5️⃣ Convertir en tenseur avec bonne forme (batch=1, seq_len, input_dim)
    X_tensor = torch.tensor(X_new_scaled, dtype=torch.float32).unsqueeze(0).to(device)

    # 6️⃣ Prédiction
    model.eval()
    with torch.no_grad():
        pred_scaled = model(X_tensor)
        pred = scaler_y.inverse_transform(pred_scaled.cpu().numpy())

    return float(pred[0][0])


seq_len = 5
seq_input = predict_df.iloc[0:seq_len].drop(columns=["label"])  # lignes 0 à 4
variation = predict_new_row(seq_input, model, selector, scaler, y_scaler, X.columns, device)
print("Prédiction:", variation)

Prédiction: 0.000774247688241303


In [14]:
df.columns

Index(['open', 'high', 'low', 'close', 'volume', 'ADX', '+DI', '-DI', 'ATR14',
       'BB_width', 'BB_pctB', 'RSI14', 'StochK', 'StochD', 'MACD',
       'MACD_signal', 'ROC', 'OBV', 'Vol_zscore', 'log_ret',
       'close_open_ratio', 'high_low_spread', 'hour', 'dayofweek', 'label',
       'open_lag1', 'open_lag2', 'open_lag3', 'open_lag4', 'open_lag5',
       'open_lag6', 'open_lag7', 'open_lag8', 'open_lag9', 'open_lag10',
       'high_lag1', 'high_lag2', 'high_lag3', 'high_lag4', 'high_lag5',
       'high_lag6', 'high_lag7', 'high_lag8', 'high_lag9', 'high_lag10',
       'low_lag1', 'low_lag2', 'low_lag3', 'low_lag4', 'low_lag5', 'low_lag6',
       'low_lag7', 'low_lag8', 'low_lag9', 'low_lag10', 'close_lag1',
       'close_lag2', 'close_lag3', 'close_lag4', 'close_lag5', 'close_lag6',
       'close_lag7', 'close_lag8', 'close_lag9', 'close_lag10', 'volume_lag1',
       'volume_lag2', 'volume_lag3', 'volume_lag4', 'volume_lag5',
       'volume_lag6', 'volume_lag7', 'volume_lag8', 'vo

In [15]:
t = 0
wallet = 10
lever = 1
fee_rate = 0.0004  # frais Binance typique
seq_len = 5
safety_pct = 0.5 / 100  # % de sécurité relatif à la variation prédite

for i in range(len(predict_df) - seq_len):
    seq_input = predict_df.iloc[i:i+seq_len].drop(columns=["label"])
    variation = predict_new_row(seq_input, model, selector, scaler, y_scaler, X.columns, device)

    if abs(variation) > 0.004:
        t += 1
        str_variation = "baisse" if variation < 0 else "monte"

        price_open = float(predict_df.iloc[i + seq_len - 1]['open'])
        price_next_close = float(predict_df.iloc[i + seq_len]['close'])
        price_high = float(predict_df.iloc[i + seq_len]['high'])
        price_low = float(predict_df.iloc[i + seq_len]['low'])

        # Écart attendu selon la variation
        expected_move = price_open * variation
        security = abs(expected_move) * safety_pct

        if variation > 0:
            target_price = price_open + expected_move - security
            # Vérifier si le target est atteint dans le High
            exit_price = min(target_price, price_high)
            if exit_price < target_price:
                exit_price = price_next_close
        else:
            target_price = price_open + expected_move + security
            exit_price = max(target_price, price_low)
            if exit_price > target_price:
                exit_price = price_next_close

        # Calcul LONG
        if variation > 0:
            position = wallet * lever
            borrowed = wallet * (lever - 1)
            units = position / price_open
            wallet = (units * exit_price) - borrowed

        # Calcul SHORT
        else:
            position = wallet * lever
            borrowed = wallet * (lever - 1)
            units = position / price_open
            sell_proceeds = units * price_open
            buy_back = units * exit_price
            wallet = wallet + sell_proceeds - buy_back - borrowed

        print(f"Step {i}, {str_variation}, prix sortie: {exit_price:.2f}, solde: {wallet:.2f}")


Step 8, baisse, prix sortie: 110246.01, solde: 9.89
Step 12, monte, prix sortie: 110376.63, solde: 9.90
Step 13, monte, prix sortie: 110233.92, solde: 9.90
Step 14, monte, prix sortie: 110360.00, solde: 9.90
Step 15, monte, prix sortie: 110450.56, solde: 9.92
Step 16, monte, prix sortie: 110246.00, solde: 9.91
Step 17, monte, prix sortie: 109733.34, solde: 9.84
Step 18, monte, prix sortie: 108795.99, solde: 9.71
Step 19, monte, prix sortie: 110220.48, solde: 9.75
Step 22, monte, prix sortie: 110785.85, solde: 9.70
Step 23, monte, prix sortie: 110914.00, solde: 9.71
Step 30, monte, prix sortie: 111339.98, solde: 9.72
Step 33, baisse, prix sortie: 111054.01, solde: 9.73
Step 34, baisse, prix sortie: 111020.68, solde: 9.76
Step 35, baisse, prix sortie: 110685.57, solde: 9.79
Step 36, baisse, prix sortie: 110724.00, solde: 9.82
Step 37, baisse, prix sortie: 111099.88, solde: 9.78
Step 38, baisse, prix sortie: 111019.99, solde: 9.76
Step 39, baisse, prix sortie: 111312.28, solde: 9.74
Step 

In [16]:
del model, X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor

In [17]:
import gc
gc.collect()

0

In [18]:
import torch
torch.cuda.empty_cache()

In [19]:
print(torch.cuda.memory_allocated() / 1024**2, "MB")
print(torch.cuda.memory_reserved() / 1024**2, "MB")

149.181640625 MB
194.0 MB
